# Intro

# Processing Steps
- Download tables from Oracle database
- Export tables to .txt files for easier manipulation in python/GIS
- Read headerData into QGIS and extract surface elevation from statewide lidar dataset (to use for calculating elevations)
- Read data into this notebook
  - Cast all columns of each dataframe/table into correct datatype
  - Clean up data
    - Header Data
      - Remove records without Lat/Lon (3,651 wells)
    - Downhole Data
      - Remove records without top and bottom information (415,337 records removed: from 3,017,085 initial records to 2,601,748)
      - Remove records without formation information (0)
      - Remove records where the Top of the record is listed as lower than the Bottom of the record (2,568 records removed)
    - wPermits
      - Remove records without bedrock depth information (7,211 wells have verified bedrock tops out of 396,524 possible wells)



# Set Up

In [70]:
#Import modules
import pandas as pd
import csv
import numpy as np

#The rest of this is just for fun/quick dataviz at the end to map and ensure seems about right
import cartopy
import matplotlib.pyplot as plt


In [71]:
#Define  filepath variables to be used later for reading/writing files
rawDirectory = '\\\\isgs-sinkhole.ad.uillinois.edu\\geophysics\\Balikian\\BedrockWellData\\Wells\\RawWellData_OracleDatabase\\TxtData\\'
processDirectory = '\\\\isgs-sinkhole.ad.uillinois.edu\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\'

downholeDataFILE = "lcl_ISGS_DOWNHOLE_DATA.txt"
headerDataFILE = "lcl_ISGS_HEADER.txt"
wpermitsFILE = "lcl_ISGS_WPERMITS.txt"

encodeType="latin-1"

#Define datatypes, to use later
downholeDataDTYPES = {'ID':np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
headerDataDTYPES = {'ID':np.uint32,'API_NUMBER':np.uint64,"TDFORMATION":str,"PRODFORM":str,"TOTAL_DEPTH":np.float64,"SECTION":np.float64,"TWP":np.float64,"TDIR":str,"RNG":np.float64,"RDIR":str,"MERIDIAN":np.float64,"FARM_NAME":str,"NSFOOT":np.float64,"NSDIR":str,"EWFOOT":np.float64,"EWDIR":str,"QUARTERS":str,"ELEVATION":np.float64,"ELEVREF":str,"COMP_DATE":str,"STATUS":str,"FARM_NUM":str,"COUNTY_CODE":np.float64,"PERMIT_NUMBER":str,"COMPANY_NAME":str,"COMPANY_CODE":str,"PERMIT_DATE":str,"CORNER":str,"LATITUDE":np.float64,"LONGITUDE":np.float64,"ENTERED_BY":str,"UPDDATE":str,"ELEVSOURCE":str}
wPermitsDTYPES = {'ID':np.uint32,"API_NUMBER":np.uint64,"PERMIT_NUMBER":str,"DRILLER_CODE":str,"OWNER_NAME":str,"OWNER_ADD1":str,"OWNER_ADD2":str,"OWNER_CITY":str,"OWNER_STATE":str,"OWNER_ZIP":str,"PERMIT_DATE":str,"WELLTYPE":str,"TOOL":str,"DEPTH":np.float64,"SAMPLE":str,"LOT":str,"SUBD":str,"ADDRESS1":str,"ADDRESS2":str,"COMMENTS":str,"IRREG":str,"SOURCE":str,"ENTERED_BY":str,"DATE_ENTERED":str,"ISWSPNUM":str,"VERIFIED_BY":str,"VERIFIED_NOTES":str,"VERIFIED_DATE":str,"BEDROCK_TOP":np.float64,"PROPERTY_ID":str,"VLQ":str,"VLC":str}


# File Read

In [72]:
#Read in downholeData file
downholeDataIN = pd.read_csv(rawDirectory+downholeDataFILE, sep=',', header='infer', encoding=encodeType)
downholeDataIN = downholeDataIN.dropna(subset=['API_NUMBER'])
downholeData = downholeDataIN.copy()
downholeDataIN

,API_NUMBER,TABLE_NAME,WHO,INTERPRET_DATE,FORMATION,THICKNESS,TOP,BOTTOM
0,1.200100e+11,FORMATION_TOPS,NaN,NaN,Cedar Valley,NaN,560.0,NaN
1,1.200100e+11,FORMATION_TOPS,NaN,NaN,Keokuk,NaN,175.0,NaN
2,1.200100e+11,FORMATION_TOPS,NaN,NaN,Kinderhook,NaN,380.0,NaN
3,1.200100e+11,FORMATION_TOPS,NaN,NaN,Pennsylvanian,NaN,5.0,NaN
4,1.200100e+11,FORMATION_TOPS,NaN,NaN,Silurian,NaN,576.0,NaN
...,...,...,...,...,...,...,...,...
3017080,4.806500e+11,FORMATION_TOPS,NaN,NaN,Platteville,NaN,217.0,NaN
3017081,4.806500e+11,FORMATION_TOPS,NaN,NaN,Quimby Mill Dolomite,NaN,213.0,NaN
3017082,4.806500e+11,FORMATION_TOPS,NaN,NaN,Spechts Ferry,NaN,211.0,NaN
3017083,4.806500e+11,FORMATION_TOPS,NaN,NaN,St James,NaN,183.0,NaN


In [73]:
#Read in headerData file
headerDataIN = pd.read_csv(rawDirectory+headerDataFILE, sep=',', header='infer', encoding=encodeType)
before = headerDataIN.shape[0]
headerDataIN = headerDataIN.dropna(subset=['LATITUDE'])
headerDataIN = headerDataIN.dropna(subset=['LONGITUDE'])
after = headerDataIN.shape[0]
print(after)
headerData = headerDataIN.copy()
headerDataIN
print(str(before-after)+' records removed without Lat/Lon')

C:\ProgramData\Anaconda3\envs\data37\lib\site-packages\IPython\core\interactiveshell.py:3524: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


634755
3651 records removed without Lat/Lon


In [74]:
#Read in wPermits file
wPermitsIN = pd.read_csv(rawDirectory+wpermitsFILE, sep=',', header='infer', encoding=encodeType)
wPermits = wPermitsIN.copy()
wPermitsIN

,API_NUMBER,PERMIT_NUMBER,DRILLER_CODE,OWNER_NAME,OWNER_ADD1,OWNER_ADD2,OWNER_CITY,OWNER_STATE,OWNER_ZIP,PERMIT_DATE,...,ENTERED_BY,DATE_ENTERED,ISWSPNUM,VERIFIED_BY,VERIFIED_NOTES,VERIFIED_DATE,BEDROCK_TOP,PROPERTY_ID,VLQ,VLC
0,120010000200,1489,NaN,"Miller, H. E. #1",NaN,NaN,NaN,NaN,NaN,8/16/1946 0:00:00,...,MEM,1/24/1995 8:55:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,120010000300,0,NaN,"Totsch, Jacob Jr.",NaN,NaN,NaN,NaN,NaN,NaN,...,MEM,2/17/1995 13:37:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120010000400,NaN,00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MEM,1/24/1995 9:29:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,120010000500,NaN,NaN,Selters #1,NaN,NaN,NaN,NaN,NaN,NaN,...,MEM,1/24/1995 9:38:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,120010000600,NaN,NaN,"Reid, Joseph #1",NaN,NaN,NaN,NaN,NaN,NaN,...,MEM,1/24/1995 9:53:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396519,480590600100,,NaN,"Aikens, Mr.",,,,,NaN,NaN,...,VJA,2/6/2006 10:12:01,NaN,ADW,NaN,1/9/2003 0:00:00,NaN,NaN,NaN,NaN
396520,480590601300,,NaN,"Zich, Tom",RR Boxholder,,Bristol,WI,NaN,NaN,...,VJA,6/19/2002 8:37:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396521,480590601700,,11061,"Perry, Leslie",RFD #3,,,,NaN,NaN,...,VJA,6/19/2002 8:48:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396522,480590602400,,NaN,"Efering, Noel",Rt. 2,,Kenosha,WI,NaN,NaN,...,VJA,6/19/2002 8:53:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Read Summary

In [75]:
print('Downhole Data has ' + str(downholeData.shape[0])+' well records.')
print("Header Data has "+str(headerData.shape[0])+" unique wells.")
print('wPermits has '+str(wPermits.shape[0])+' other potential wells with bedrock data points (only about '+str(wPermits['BEDROCK_TOP'].notna().sum())+' with verified bedrock tops will be used)')

Downhole Data has 3017085 well records.
Header Data has 634755 unique wells.
wPermits has 396524 other potential wells with bedrock data points (only about 7211 with verified bedrock tops will be used)


# Read Cleanup

## First cast data to proper datatypes

These datatypes were defined in the second code cell

In [76]:
#Recast all columns to datatypes of downholeData to previously defined types
for i in range(0, np.shape(downholeData)[1]):
    downholeData.iloc[:,i] = downholeDataIN.iloc[:,i].astype(downholeDataDTYPES[downholeDataIN.iloc[:,i].name])
downholeData

,API_NUMBER,TABLE_NAME,WHO,INTERPRET_DATE,FORMATION,THICKNESS,TOP,BOTTOM
0,120010000200,FORMATION_TOPS,nan,nan,Cedar Valley,NaN,560.0,NaN
1,120010000200,FORMATION_TOPS,nan,nan,Keokuk,NaN,175.0,NaN
2,120010000200,FORMATION_TOPS,nan,nan,Kinderhook,NaN,380.0,NaN
3,120010000200,FORMATION_TOPS,nan,nan,Pennsylvanian,NaN,5.0,NaN
4,120010000200,FORMATION_TOPS,nan,nan,Silurian,NaN,576.0,NaN
...,...,...,...,...,...,...,...,...
3017080,480650003600,FORMATION_TOPS,nan,nan,Platteville,NaN,217.0,NaN
3017081,480650003600,FORMATION_TOPS,nan,nan,Quimby Mill Dolomite,NaN,213.0,NaN
3017082,480650003600,FORMATION_TOPS,nan,nan,Spechts Ferry,NaN,211.0,NaN
3017083,480650003600,FORMATION_TOPS,nan,nan,St James,NaN,183.0,NaN


In [77]:
#Recast all columns to datatypes of headerData to previously defined types
for i in range(0, np.shape(headerData)[1]):
    headerData.iloc[:,i] = headerDataIN.iloc[:,i].astype(headerDataDTYPES[headerDataIN.iloc[:,i].name])
headerData

,API_NUMBER,TDFORMATION,PRODFORM,TOTAL_DEPTH,SECTION,TWP,TDIR,RNG,RDIR,MERIDIAN,...,PERMIT_NUMBER,COMPANY_NAME,COMPANY_CODE,PERMIT_DATE,CORNER,LATITUDE,LONGITUDE,ENTERED_BY,UPDDATE,ELEVSOURCE
0,120352191101,353SLEM,nan,1400.0,15.0,10.0,N,10.0,E,3.0,...,6650,"Traub, John",25240.0,11/6/1978 0:00:00,nan,39.306969,-88.078211,JET,7/23/1998 11:10:20,nan
1,120352194900,302GENV,nan,4183.0,27.0,9.0,N,9.0,E,3.0,...,5115,"LMC Resources, Inc.",15432.0,12/21/1977 0:00:00,nan,39.188754,-88.186257,BEL,5/11/2005 0:00:00,Company
2,120352204200,309DVNN,nan,2140.0,18.0,10.0,N,11.0,E,3.0,...,6280,Partlow & Cochonour,18150.0,8/31/1978 0:00:00,nan,39.318753,-88.013434,BEL,5/13/2005 0:00:00,Company
3,120352204300,309DVNN,nan,4175.0,34.0,10.0,N,9.0,E,3.0,...,6640,C. E. Brehm Drilling & Prod.,6980.0,11/3/1978 0:00:00,nan,39.272700,-88.176757,BEL,5/12/2005 0:00:00,Company
4,120352207500,353SLEM,nan,2461.0,34.0,9.0,N,10.0,E,3.0,...,8134,"Hagen, F. Lowell",12542.0,2/10/1981 0:00:00,nan,39.178963,-88.077840,BEL,8/18/2014 14:05:50,Company
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638400,120810182400,nan,nan,24.0,3.0,4.0,S,4.0,E,3.0,...,nan,PEABODY COAL CO.,nan,nan,nan,38.198542,-88.744783,nan,nan,nan
638401,120810182500,nan,nan,59.0,25.0,3.0,S,3.0,E,3.0,...,nan,PEABODY COAL CO.,nan,nan,nan,38.239806,-88.816906,nan,nan,nan
638403,120810244500,nan,nan,711.0,9.0,4.0,S,2.0,E,3.0,...,nan,FREEMAN UNITED,nan,nan,nan,38.196151,-88.997197,nan,nan,nan
638404,120810245100,nan,nan,518.0,20.0,3.0,S,1.0,E,3.0,...,nan,FREEMAN UNITED,nan,nan,nan,38.253604,-89.128477,nan,nan,nan


In [78]:
#Recast all columns to datatypes of wPermits to previously defined types
for i in range(0, np.shape(wPermits)[1]):
    wPermits.iloc[:,i] = wPermitsIN.iloc[:,i].astype(wPermitsDTYPES[wPermitsIN.iloc[:,i].name])
wPermits

,API_NUMBER,PERMIT_NUMBER,DRILLER_CODE,OWNER_NAME,OWNER_ADD1,OWNER_ADD2,OWNER_CITY,OWNER_STATE,OWNER_ZIP,PERMIT_DATE,...,ENTERED_BY,DATE_ENTERED,ISWSPNUM,VERIFIED_BY,VERIFIED_NOTES,VERIFIED_DATE,BEDROCK_TOP,PROPERTY_ID,VLQ,VLC
0,120010000200,1489,nan,"Miller, H. E. #1",nan,nan,nan,nan,nan,8/16/1946 0:00:00,...,MEM,1/24/1995 8:55:50,nan,nan,nan,nan,NaN,nan,nan,nan
1,120010000300,0,nan,"Totsch, Jacob Jr.",nan,nan,nan,nan,nan,nan,...,MEM,2/17/1995 13:37:11,nan,nan,nan,nan,NaN,nan,nan,nan
2,120010000400,nan,00,nan,nan,nan,nan,nan,nan,nan,...,MEM,1/24/1995 9:29:56,nan,nan,nan,nan,NaN,nan,nan,nan
3,120010000500,nan,nan,Selters #1,nan,nan,nan,nan,nan,nan,...,MEM,1/24/1995 9:38:16,nan,nan,nan,nan,NaN,nan,nan,nan
4,120010000600,nan,nan,"Reid, Joseph #1",nan,nan,nan,nan,nan,nan,...,MEM,1/24/1995 9:53:03,nan,nan,nan,nan,NaN,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396519,480590600100,,nan,"Aikens, Mr.",,,,,nan,nan,...,VJA,2/6/2006 10:12:01,nan,ADW,nan,1/9/2003 0:00:00,NaN,nan,nan,nan
396520,480590601300,,nan,"Zich, Tom",RR Boxholder,,Bristol,WI,nan,nan,...,VJA,6/19/2002 8:37:42,nan,nan,nan,nan,NaN,nan,nan,nan
396521,480590601700,,11061,"Perry, Leslie",RFD #3,,,,nan,nan,...,VJA,6/19/2002 8:48:05,nan,nan,nan,nan,NaN,nan,nan,nan
396522,480590602400,,nan,"Efering, Noel",Rt. 2,,Kenosha,WI,nan,nan,...,VJA,6/19/2002 8:53:06,nan,nan,nan,nan,NaN,nan,nan,nan


## Then get rid of extra columns to save time and memory

In [79]:
#Desigate which columns to keep from here on out
downhole_keepCols = ["API_NUMBER","TABLE_NAME","FORMATION","THICKNESS","TOP","BOTTOM"]
headers_keepCols = ['API_NUMBER',"TOTAL_DEPTH","SECTION","TWP","TDIR","RNG","RDIR","MERIDIAN","QUARTERS","ELEVATION","ELEVREF","COUNTY_CODE","LATITUDE","LONGITUDE","ELEVSOURCE"]
wPermits_keepCols = ["API_NUMBER","WELLTYPE","DEPTH","ISWSPNUM","VERIFIED_BY","VERIFIED_NOTES","VERIFIED_DATE","BEDROCK_TOP","VLQ","VLC"]

In [80]:
downholeData = downholeData[downhole_keepCols]
headerData = headerData[headers_keepCols]
wPermits = wPermits[wPermits_keepCols]
headerData

,API_NUMBER,TOTAL_DEPTH,SECTION,TWP,TDIR,RNG,RDIR,MERIDIAN,QUARTERS,ELEVATION,ELEVREF,COUNTY_CODE,LATITUDE,LONGITUDE,ELEVSOURCE
0,120352191101,1400.0,15.0,10.0,N,10.0,E,3.0,SW SW,611.0,GL,35.0,39.306969,-88.078211,nan
1,120352194900,4183.0,27.0,9.0,N,9.0,E,3.0,SE SW,584.0,KB,35.0,39.188754,-88.186257,Company
2,120352204200,2140.0,18.0,10.0,N,11.0,E,3.0,NE NE,652.0,KB,35.0,39.318753,-88.013434,Company
3,120352204300,4175.0,34.0,10.0,N,9.0,E,3.0,NE NE,592.0,KB,35.0,39.272700,-88.176757,Company
4,120352207500,2461.0,34.0,9.0,N,10.0,E,3.0,NW SW,582.0,GL,35.0,39.178963,-88.077840,Company
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638400,120810182400,24.0,3.0,4.0,S,4.0,E,3.0,SESE,455.0,nan,81.0,38.198542,-88.744783,nan
638401,120810182500,59.0,25.0,3.0,S,3.0,E,3.0,SENE,528.0,nan,81.0,38.239806,-88.816906,nan
638403,120810244500,711.0,9.0,4.0,S,2.0,E,3.0,NENW,400.0,nan,81.0,38.196151,-88.997197,nan
638404,120810245100,518.0,20.0,3.0,S,1.0,E,3.0,NW,512.0,nan,81.0,38.253604,-89.128477,nan


## Export data so we don't have to start from scratch each session

In [81]:
headerOutFile = 'headerData__essCols.csv'
downholeOutFile = 'downholeData_essCols.csv'
wPermitsOutFile = 'wPermits_essCols.csv'

encodeType='latin-1'

processDirectory = '\\\\isgs-sinkhole.ad.uillinois.edu\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\'

In [82]:
headerData.reset_index(inplace=True, drop=True)
downholeData.reset_index(inplace=True, drop=True)
wPermits.reset_index(inplace=True, drop=True)

In [83]:
headerData.to_csv(processDirectory+headerOutFile,index_label='ID')
downholeData.to_csv(processDirectory+downholeOutFile,index_label='ID')
wPermits.to_csv(processDirectory+wPermitsOutFile,index_label='ID')

# Data Cleaning

## First, let's clean up records in the data without the necessary information

First, the headerfileData had elevations from the 10m statewide DEM added to get a consistent surface elevation data source (I would like to do this with lidar data, but that is much more involved)

Start by reading the files back in again

In [84]:
headerInFile = 'headerData__essCols_elev10mDTM.csv'
downholeInFile = 'downholeData_essCols.csv'
wPermitsInFile = 'wPermits_essCols.csv'

encodeType='latin-1'

processDirectory = '\\\\isgs-sinkhole.ad.uillinois.edu\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\'

headers_useCols = ['API_NUMBER',"TOTAL_DEPTH","SECTION","TWP","TDIR","RNG","RDIR","MERIDIAN","QUARTERS","ELEVATION","ELEVREF","COUNTY_CODE","LATITUDE","LONGITUDE","ELEVSOURCE", "10mDTM_ft"]
downhole_useCols = ["API_NUMBER","TABLE_NAME","FORMATION","THICKNESS","TOP","BOTTOM"]
wPermits_useCols = ["API_NUMBER","WELLTYPE","DEPTH","ISWSPNUM","VERIFIED_BY","VERIFIED_NOTES","VERIFIED_DATE","BEDROCK_TOP","VLQ","VLC"]

headerDataIN = pd.read_csv(processDirectory+headerInFile, sep=',', header='infer', encoding=encodeType, usecols=headers_useCols)
downholeDataIN = pd.read_csv(processDirectory+downholeInFile, sep=',', header='infer', encoding=encodeType, usecols=downhole_useCols)
wPermitsIN = pd.read_csv(processDirectory+wPermitsInFile, sep=',', header='infer', encoding=encodeType, usecols=wPermits_useCols)

In [85]:
headerData = headerDataIN.copy()
downholeData = downholeDataIN.copy()
wPermits = wPermitsIN.copy()

Define datatypes for all columns

In [86]:
downholeDataDTYPES = {"ID":np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
headerDataDTYPES = {"ID":np.uint32, 'API_NUMBER':np.uint64,"TDFORMATION":str,"PRODFORM":str,"TOTAL_DEPTH":np.float64,"SECTION":np.float64,"TWP":np.float64,"TDIR":str,"RNG":np.float64,"RDIR":str,"MERIDIAN":np.float64,"FARM_NAME":str,"NSFOOT":np.float64,"NSDIR":str,"EWFOOT":np.float64,"EWDIR":str,"QUARTERS":str,"ELEVATION":np.float64,"ELEVREF":str,"COMP_DATE":str,"STATUS":str,"FARM_NUM":str,"COUNTY_CODE":np.float64,"PERMIT_NUMBER":str,"COMPANY_NAME":str,"COMPANY_CODE":str,"PERMIT_DATE":str,"CORNER":str,"LATITUDE":np.float64,"LONGITUDE":np.float64,"ENTERED_BY":str,"UPDDATE":str,"ELEVSOURCE":str, "10mDTM_ft":np.float64}
wPermitsDTYPES = {"ID":np.uint32, "API_NUMBER":np.uint64,"PERMIT_NUMBER":str,"DRILLER_CODE":str,"OWNER_NAME":str,"OWNER_ADD1":str,"OWNER_ADD2":str,"OWNER_CITY":str,"OWNER_STATE":str,"OWNER_ZIP":str,"PERMIT_DATE":str,"WELLTYPE":str,"TOOL":str,"DEPTH":np.float64,"SAMPLE":str,"LOT":str,"SUBD":str,"ADDRESS1":str,"ADDRESS2":str,"COMMENTS":str,"IRREG":str,"SOURCE":str,"ENTERED_BY":str,"DATE_ENTERED":str,"ISWSPNUM":str,"VERIFIED_BY":str,"VERIFIED_NOTES":str,"VERIFIED_DATE":str,"BEDROCK_TOP":np.float64,"PROPERTY_ID":str,"VLQ":str,"VLC":str}

#Recast all columns to datatypes of downholeData to previously defined types
for i in range(0, np.shape(downholeData)[1]):
    downholeData.iloc[:,i] = downholeDataIN.iloc[:,i].astype(downholeDataDTYPES[downholeDataIN.iloc[:,i].name])
    
#Recast all columns to datatypes of headerData to previously defined types
for i in range(0, np.shape(headerData)[1]):
    headerData.iloc[:,i] = headerDataIN.iloc[:,i].astype(headerDataDTYPES[headerDataIN.iloc[:,i].name])

#Recast all columns to datatypes of wPermits to previously defined types
for i in range(0, np.shape(wPermits)[1]):
    wPermits.iloc[:,i] = wPermitsIN.iloc[:,i].astype(wPermitsDTYPES[wPermitsIN.iloc[:,i].name])


Remove headerData rows without surface elevation information (this also clips data from outside Illinois)

In [87]:
before = headerData.shape[0]
print("Number of rows before dropping those without surface elevation information: "+str(headerData.shape[0]))
headerData['10mDTM_ft'].replace('',np.nan,inplace=True)
headerData.dropna(subset=['10mDTM_ft'],inplace=True)
print("Number of rows after dropping those without surface elevation information: "+str(headerData.shape[0]))
print('Well records deleted: '+str(before-headerData.shape[0]))


Number of rows before dropping those without surface elevation information: 634755
Number of rows after dropping those without surface elevation information: 634533
Well records deleted: 222


Remove rows from downholeData with no depth information and where depth information is obviously bad (i.e., top depth > bottom depth)

In [88]:
#Replace empty cells in top/bottom columns with nans
downholeData['TOP'] = downholeData['TOP'].replace('', np.nan)
downholeData['BOTTOM'] = downholeData['BOTTOM'].replace('', np.nan)
before = downholeData.shape[0]
print("Number of rows before dropping those without record depth information: "+str(downholeDataIN.shape[0]))

#Drop records without depth information
downholeData = downholeData.dropna(subset=['TOP'])
downholeData = downholeData.dropna(subset=['BOTTOM'])
print("Number of rows after dropping those without record depth information: "+str(downholeData.shape[0]))
print('Well records deleted: '+str(before-downholeData.shape[0]))
before = downholeData.shape[0]

#Replace empty cells in formation column with nans
downholeData['FORMATION'] = downholeData['FORMATION'].replace('', np.nan) 
before = downholeData.shape[0]
print("Number of rows before dropping those without FORMATION information: "+str(downholeData.shape[0]))

#Drop records without FORMATION information
downholeData = downholeData.dropna(subset=['FORMATION'])
print("Number of rows after dropping those without FORMATION information: "+str(downholeData.shape[0]))
print('Well records deleted: '+str(before-downholeData.shape[0]))

#Calculate thickness (replaces existing column) and drop records with obviously bad depth information
downholeData['THICKNESS'] = downholeData['BOTTOM'] - downholeData['TOP']
before = downholeData.shape[0]
downholeData = downholeData[(downholeData['THICKNESS'] >= 0)]
print("Number of rows before dropping those with obviously bad depth information: "+str(before))
print("Number of rows after dropping those with obviously bad depth information: "+str(downholeData.shape[0]))
print('Well records deleted: '+str(before-downholeData.shape[0]))
downholeData

Number of rows before dropping those without record depth information: 3017085
Number of rows after dropping those without record depth information: 2601748
Well records deleted: 415337
Number of rows before dropping those without FORMATION information: 2601748
Number of rows after dropping those without FORMATION information: 2601748
Well records deleted: 0
Number of rows before dropping those with obviously bad depth information: 2601748
Number of rows after dropping those with obviously bad depth information: 2599180
Well records deleted: 2568


,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM
6,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0
7,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0
8,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0
14,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0
15,120010000800,WFORMATIONS,interval,175.0,0.0,175.0
...,...,...,...,...,...,...
3017041,480590094700,WFORMATIONS,clay gravel @ sand,210.0,15.0,225.0
3017042,480590094700,WFORMATIONS,gravel @ clay,150.0,10.0,160.0
3017043,480590094700,WFORMATIONS,gravel @ clay,170.0,5.0,175.0
3017044,480590094700,WFORMATIONS,hardpan,100.0,50.0,150.0


## Now, let's only hold onto the wPermits records with bedrock data

In [89]:
#Replace empty cells in bedrocktop column with nans
wPermits['BEDROCK_TOP'] = wPermits['BEDROCK_TOP'].replace('', np.nan)
before = wPermits.shape[0]
print("Number of rows before dropping those without record depth information: "+str(before))

#Drop records without depth information
wPermits = wPermits.dropna(subset=['BEDROCK_TOP'])
wPermits = wPermits[wPermits['BEDROCK_TOP']<99999]
print("Number of rows after dropping those without record depth information: "+str(wPermits.shape[0]))
print('Well records deleted: '+str(before-wPermits.shape[0]))

Number of rows before dropping those without record depth information: 396524
Number of rows after dropping those without record depth information: 6061
Well records deleted: 390463


Now we are going to export this data, again to save memory and space and so we do not lose data

In [90]:
headerOutFile = 'headerData__essCols_elev10mClip.csv'
downholeOutFile = 'downholeData_essCols_depthNA.csv'
wPermitsOutFile = 'wPermits_essCols_BRdepthNA.csv'

encodeType='latin-1'

processDirectory = '\\\\isgs-sinkhole.ad.uillinois.edu\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\'

In [93]:
headerData.reset_index(inplace=True,drop=True)
downholeData.reset_index(inplace=True,drop=True)
wPermits.reset_index(inplace=True,drop=True)

headerData.to_csv(processDirectory+headerOutFile,index_label='ID')
downholeData.to_csv(processDirectory+downholeOutFile,index_label='ID')
wPermits.to_csv(processDirectory+wPermitsOutFile,index_label='ID')

# Data Merging

First, read data back in again

In [94]:
#File names
headerInFile = 'headerData__essCols_elev10mClip.csv'
downholeInFile = 'downholeData_essCols_depthNA.csv'
wPermitsInFile = 'wPermits_essCols_BRdepthNA.csv'

encodeType='latin-1'

#Directory path
processDirectory = '\\\\isgs-sinkhole.ad.uillinois.edu\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\'

#Columns to use
headers_useCols = ['ID','API_NUMBER',"TOTAL_DEPTH","SECTION","TWP","TDIR","RNG","RDIR","MERIDIAN","QUARTERS","ELEVATION","ELEVREF","COUNTY_CODE","LATITUDE","LONGITUDE","ELEVSOURCE"]
downhole_useCols = ["ID","API_NUMBER","TABLE_NAME","FORMATION","THICKNESS","TOP","BOTTOM"]
wPermits_useCols = ['ID',"API_NUMBER","WELLTYPE","DEPTH","ISWSPNUM","VERIFIED_BY","VERIFIED_NOTES","VERIFIED_DATE","BEDROCK_TOP","VLQ","VLC"]

#Read it in
headerDataIN = pd.read_csv(processDirectory+headerInFile, sep=',', header='infer', encoding=encodeType, index_col='ID',usecols=headers_useCols)
downholeDataIN = pd.read_csv(processDirectory+downholeInFile, sep=',', header='infer', encoding=encodeType, index_col='ID',usecols=downhole_useCols)
wPermitsIN = pd.read_csv(processDirectory+wPermitsInFile, sep=',', header='infer', encoding=encodeType, index_col='ID',usecols=wPermits_useCols)

#Make a copy, in case we need to refer back to original read-in data
headerData = headerDataIN.copy()
downholeData = downholeDataIN.copy()
wPermits = wPermitsIN.copy()

In [95]:
downholeDataDTYPES = {"ID":np.uint32, "API_NUMBER":np.uint64,"TABLE_NAME":str,"WHO":str,"INTERPRET_DATE":str,"FORMATION":str,"THICKNESS":np.float64,"TOP":np.float64,"BOTTOM":np.float64}
headerDataDTYPES = {"ID":np.uint32, 'API_NUMBER':np.uint64,"TDFORMATION":str,"PRODFORM":str,"TOTAL_DEPTH":np.float64,"SECTION":np.float64,"TWP":np.float64,"TDIR":str,"RNG":np.float64,"RDIR":str,"MERIDIAN":np.float64,"FARM_NAME":str,"NSFOOT":np.float64,"NSDIR":str,"EWFOOT":np.float64,"EWDIR":str,"QUARTERS":str,"ELEVATION":np.float64,"ELEVREF":str,"COMP_DATE":str,"STATUS":str,"FARM_NUM":str,"COUNTY_CODE":np.float64,"PERMIT_NUMBER":str,"COMPANY_NAME":str,"COMPANY_CODE":str,"PERMIT_DATE":str,"CORNER":str,"LATITUDE":np.float64,"LONGITUDE":np.float64,"ENTERED_BY":str,"UPDDATE":str,"ELEVSOURCE":str}
wPermitsDTYPES = {"ID":np.uint32, "API_NUMBER":np.uint64,"PERMIT_NUMBER":str,"DRILLER_CODE":str,"OWNER_NAME":str,"OWNER_ADD1":str,"OWNER_ADD2":str,"OWNER_CITY":str,"OWNER_STATE":str,"OWNER_ZIP":str,"PERMIT_DATE":str,"WELLTYPE":str,"TOOL":str,"DEPTH":np.float64,"SAMPLE":str,"LOT":str,"SUBD":str,"ADDRESS1":str,"ADDRESS2":str,"COMMENTS":str,"IRREG":str,"SOURCE":str,"ENTERED_BY":str,"DATE_ENTERED":str,"ISWSPNUM":str,"VERIFIED_BY":str,"VERIFIED_NOTES":str,"VERIFIED_DATE":str,"BEDROCK_TOP":np.float64,"PROPERTY_ID":str,"VLQ":str,"VLC":str}

#Recast all columns to datatypes of downholeData to previously defined types
for i in range(0, np.shape(downholeData)[1]):
    downholeData.iloc[:,i] = downholeDataIN.iloc[:,i].astype(downholeDataDTYPES[downholeDataIN.iloc[:,i].name])
    
#Recast all columns to datatypes of headerData to previously defined types
for i in range(0, np.shape(headerData)[1]):
    headerData.iloc[:,i] = headerDataIN.iloc[:,i].astype(headerDataDTYPES[headerDataIN.iloc[:,i].name])

#Recast all columns to datatypes of wPermits to previously defined types
for i in range(0, np.shape(wPermits)[1]):
    wPermits.iloc[:,i] = wPermitsIN.iloc[:,i].astype(wPermitsDTYPES[wPermitsIN.iloc[:,i].name])

In [96]:
downholeData.shape[0]

2599180

# Processing

The following flags are used to mark the classification method:
- 0: Not classified
- 1: Specific Search Term Match
- 2: wPermits bedrock top pick
- 3: Intervals >550' below ground surface
- 4: Wildcard match (startTerm) - no context
- 5: Wildcard match (startTerm) - with context
- Top of well?

In [97]:
#Read in dictionary files for downhole data
dictDir = "\\\\isgs-sinkhole.ad.uillinois.edu\\geophysics\\Balikian\\BedrockWellData\\Scripts\\FormationDicts\\"

specTermsFile = "SearchTerms-Specific_BedrockOrNo.csv" #Specific matches
startTermsFile = "SearchTerms-Start_BedrockOrNo.csv" #Wildcard matches for the start of the description

In [98]:
specTerms = pd.read_csv(dictDir+specTermsFile, index_col='ID')
startTerms = pd.read_csv(dictDir+startTermsFile, index_col='ID')
specTerms.rename(columns={'SearchTerm':'FORMATION', 'InterpUpdate':'INTERPRETATION'}, inplace=True)
specTerms['CLASS_FLAG'] = 1
startTerms.rename(columns={'StartTerm':'FORMATION', 'InterpUpdate':'INTERPRETATION'}, inplace=True)
startTerms['CLASS_FLAG'] = 4
specTerms

,FORMATION,INTERPRETATION,CLASS_FLAG
ID,,,
0,"sand, fine gr",SAND,1
1,"sand, fine-large gr",SAND,1
2,"sand, some large gr",SAND,1
3,toop soil,SOIL,1
4,grav in muck,GRAVEL,1
...,...,...,...
1936,yellow sandstone at,BEDROCK,1
1937,yellow sandstone/clay,BEDROCK AND OTHER,1
1938,yellow St. Peter,BEDROCK,1


In [99]:
specTermsDtypes = {'FORMATION':str,'INTERPRETATION':str, 'CLASS_FLAG':np.uint8}
startTermsDtypes = {'FORMATION':str,'INTERPRETATION':str, 'CLASS_FLAG':np.uint8}

#Recast all columns to datatypes of headerData to previously defined types
for i in range(0, np.shape(specTerms)[1]):
    specTerms.iloc[:,i] = specTerms.iloc[:,i].astype(specTermsDtypes[specTerms.iloc[:,i].name])
    startTerms.iloc[:,i] = startTerms.iloc[:,i].astype(startTermsDtypes[startTerms.iloc[:,i].name])
specTerms.drop_duplicates(subset='FORMATION',inplace=True) #Apparently, there are some duplicate definitions, which need to be deleted first
startTerms.drop_duplicates(subset='FORMATION',inplace=True)
specTerms.reset_index(inplace=True, drop=True)
startTerms.reset_index(inplace=True, drop=True)


In [100]:
downholeData_Interps = pd.merge(downholeData, specTerms.set_index('FORMATION'), on='FORMATION',how='left')
downholeData_Interps['BEDROCK_FLAG'] = downholeData_Interps['INTERPRETATION'] == 'BEDROCK'
print("Records Classified with full search term: "+str(int(downholeData_Interps['CLASS_FLAG'].sum())))
print("Records Classified with full search term: "+str(round((downholeData_Interps['CLASS_FLAG'].sum()/downholeData_Interps.shape[0])*100,2))+"% of data")
downholeData_Interps

Records Classified with full search term: 1165812
Records Classified with full search term: 44.85% of data


,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,BEDROCK_FLAG
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,NaN,False
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,NaN,False
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,NaN,False
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,NaN,False
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
2599175,480590094700,WFORMATIONS,clay gravel @ sand,210.0,15.0,225.0,NaN,NaN,False
2599176,480590094700,WFORMATIONS,gravel @ clay,150.0,10.0,160.0,NaN,NaN,False
2599177,480590094700,WFORMATIONS,gravel @ clay,170.0,5.0,175.0,NaN,NaN,False
2599178,480590094700,WFORMATIONS,hardpan,100.0,50.0,150.0,SOIL,1.0,False


In [101]:
specDF = downholeData_Interps[downholeData_Interps['CLASS_FLAG'].notna()]
specDF

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,BEDROCK_FLAG
5,120010000900,WFORMATIONS,coal,2.0,133.0,135.0,BEDROCK,1.0,True
6,120010000900,WFORMATIONS,fire clay,1.0,135.0,136.0,BEDROCK,1.0,True
9,120010001000,WFORMATIONS,fire clay,2.0,137.0,139.0,BEDROCK,1.0,True
16,120010001200,WFORMATIONS,black slate,1.0,35.0,36.0,BEDROCK,1.0,True
17,120010001200,WFORMATIONS,coal,1.0,36.0,37.0,BEDROCK,1.0,True
...,...,...,...,...,...,...,...,...,...
2599172,480590083600,WFORMATIONS,hardpan,189.0,21.0,210.0,SOIL,1.0,False
2599173,480590083600,WFORMATIONS,limestone,235.0,23.0,258.0,BEDROCK,1.0,True
2599174,480590094700,WFORMATIONS,clay,0.0,100.0,100.0,CLAY,1.0,False
2599178,480590094700,WFORMATIONS,hardpan,100.0,50.0,150.0,SOIL,1.0,False


In [102]:
searchDF = downholeData_Interps[downholeData_Interps['CLASS_FLAG'].isna()]
searchDF

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,BEDROCK_FLAG
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,NaN,False
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,NaN,False
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,NaN,False
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,NaN,False
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
2599170,480590083600,WFORMATIONS,clay @ gravel,53.0,136.0,189.0,NaN,NaN,False
2599171,480590083600,WFORMATIONS,dark brown shale,210.0,25.0,235.0,NaN,NaN,False
2599175,480590094700,WFORMATIONS,clay gravel @ sand,210.0,15.0,225.0,NaN,NaN,False
2599176,480590094700,WFORMATIONS,gravel @ clay,150.0,10.0,160.0,NaN,NaN,False


In [103]:
for i,s in enumerate(startTerms['FORMATION']):
    searchDF['CLASS_FLAG'].where(~searchDF['FORMATION'].str.startswith(s,na=False),4,inplace=True)
    searchDF['INTERPRETATION'].where(~searchDF['FORMATION'].str.startswith(s,na=False),startTerms.loc[i,'INTERPRETATION'],inplace=True)
searchDF['BEDROCK_FLAG'] = searchDF["INTERPRETATION"] == 'BEDROCK'

C:\ProgramData\Anaconda3\envs\data37\lib\site-packages\pandas\core\generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\ProgramData\Anaconda3\envs\data37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [104]:
print("Records classified with start search term: "+str(int(searchDF['CLASS_FLAG'].count())))
print("Records classified with start search term: "+str(round((searchDF['CLASS_FLAG'].count()/searchDF.shape[0])*100,2))+"% of remaining data")
print("Records classified with both search terms: "+str(round(((searchDF['CLASS_FLAG'].count()+specDF['CLASS_FLAG'].count())/downholeData_Interps.shape[0])*100,2))+"% of all data")

Records classified with start search term: 225608
Records classified with start search term: 15.74% of remaining data
Records classified with both search terms: 53.53% of all data


In [105]:
searchDF['BEDROCK_FLAG'].value_counts()

False    1306360
True      127008
Name: BEDROCK_FLAG, dtype: int64

In [106]:
searchDF.INTERPRETATION.value_counts()

BEDROCK                127008
CLAY                    79105
SAND AND GRAVEL          7473
SAND                     5677
SOIL                     1633
SILT                     1285
FILL                     1117
BOULDERY MATERIAL         705
ORGANIC MATERIAL          398
CLAY AND GRAVEL           275
GENERIC                   236
SAND AND CLAY MIX         161
SILT AND SAND             146
GRAVEL                     93
SILT AND SAND MIX          87
CLAY WITH SAND             80
CLAY AND SAND MIX          80
BEDROCK AND OTHER          22
TOPSOIL                    12
MUD AND GRAVEL             11
MUD AND SAND                2
GRAVEL WITH SAND            1
SILT AND GRAVEL MIX         1
Name: INTERPRETATION, dtype: int64

In [107]:
searchDF.iloc[100:120,:]

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,BEDROCK_FLAG
128,120010002400,WFORMATIONS,ironstone,1.0,115.0,116.0,NaN,NaN,False
129,120010002400,WFORMATIONS,"sandstone,soft",50.0,65.0,115.0,BEDROCK,4.0,True
130,120010002500,WFORMATIONS,blue clay,60.0,60.0,120.0,CLAY,4.0,False
136,120010002700,WFORMATIONS,coal no.2,3.0,55.0,58.0,NaN,NaN,False
138,120010002700,WFORMATIONS,interval,37.0,0.0,37.0,NaN,NaN,False
141,120010002900,WFORMATIONS,"c w/ucl,sft,s,lgt gry",3.0,35.0,38.0,NaN,NaN,False
142,120010002900,WFORMATIONS,"s,most f,slgt sty,noncalc,yl-brn",2.0,13.0,15.0,NaN,NaN,False
143,120010002900,WFORMATIONS,"s,most f,sty,noncalc,yl-brn",3.0,10.0,13.0,NaN,NaN,False
144,120010002900,WFORMATIONS,"s,most f,vy ltl st,noncalc,ylsh",2.0,18.0,20.0,NaN,NaN,False
145,120010002900,WFORMATIONS,"s,most f,vy ltl st,noncalc,ylsh",3.0,15.0,18.0,NaN,NaN,False


Merge data back together

In [108]:
downholeData_Cat = pd.concat([specDF,searchDF], join='inner').sort_index()
downholeData_Cat

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,BEDROCK_FLAG
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,NaN,False
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,NaN,False
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,NaN,False
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,NaN,False
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
2599175,480590094700,WFORMATIONS,clay gravel @ sand,210.0,15.0,225.0,NaN,NaN,False
2599176,480590094700,WFORMATIONS,gravel @ clay,150.0,10.0,160.0,NaN,NaN,False
2599177,480590094700,WFORMATIONS,gravel @ clay,170.0,5.0,175.0,NaN,NaN,False
2599178,480590094700,WFORMATIONS,hardpan,100.0,50.0,150.0,SOIL,1.0,False


In [109]:
downholeData = downholeData_Cat.copy()

## Classify all data deeper than 550'

First, separate into data that still needs to be classified and data that already has been classified

In [110]:
searchDF = downholeData[downholeData['CLASS_FLAG'].isna()] #Data that has not yet been classified
foundDF = downholeData[downholeData['CLASS_FLAG'].notna()] #Data that has already been classified
searchDF

,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,BEDROCK_FLAG
0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,NaN,False
1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,NaN,False
2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,NaN,False
3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,NaN,False
4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
2599170,480590083600,WFORMATIONS,clay @ gravel,53.0,136.0,189.0,NaN,NaN,False
2599171,480590083600,WFORMATIONS,dark brown shale,210.0,25.0,235.0,NaN,NaN,False
2599175,480590094700,WFORMATIONS,clay gravel @ sand,210.0,15.0,225.0,NaN,NaN,False
2599176,480590094700,WFORMATIONS,gravel @ clay,150.0,10.0,160.0,NaN,NaN,False


In [111]:
searchDF['CLASS_FLAG'].mask(searchDF['TOP']>550,3,inplace=True) #Add a Classification Flag of 3 (bedrock b/c it's deepter than 550') to all records where the top of the interval is >550'
searchDF['BEDROCK_FLAG'].mask(searchDF['TOP']>550,True,inplace=True)
###COULD PROBABLY DO THIS FOR BOTTOM TOO!!!!????

prevClass = downholeData['CLASS_FLAG'].value_counts().sum()
brDepthClass = searchDF['CLASS_FLAG'].value_counts()[3.0]
total = downholeData.shape[0]

newPercClass = round((brDepthClass/(total-prevClass))*100, 2)

print("Records classified deeper than 550': " + str(brDepthClass))
print("Records classified deeper than 550', as a percentage of remaining unclassified records: " + str(newPercClass)+"%")
print("Total bedrock records classified with search terms and depth: " + str(prevClass+brDepthClass))
print("This is now a total of "+str(round((prevClass+brDepthClass)*100/downholeData.shape[0],2))+"% of the data classified.")



Records classified deeper than 550': 273604
Records classified deeper than 550', as a percentage of remaining unclassified records: 22.65%
Total bedrock records classified with search terms and depth: 1665024
This is now a total of 64.06% of the data classified.


C:\ProgramData\Anaconda3\envs\data37\lib\site-packages\pandas\core\generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Put the classified and unclassified data back together again

In [112]:
downholeData_Cat = pd.concat([foundDF,searchDF], join='inner').sort_index()

In [113]:
downholeData = downholeData_Cat.copy() #make a copy of the data to continue further work on

Add '0' flag for data still not classified

In [114]:
downholeData['CLASS_FLAG'].fillna(0, inplace=True)

In [115]:
downholeData['CLASS_FLAG'].value_counts()

1.0    1165812
0.0     934156
3.0     273604
4.0     225608
Name: CLASS_FLAG, dtype: int64

## Find First Bedrock Record

Find all unique wells in downhole dataset

In [116]:
#Get Unique well APIs
uniqueWells = downholeData['API_NUMBER'].unique()
wellsDF = pd.DataFrame(uniqueWells)
print('Number of unique wells in downholeData: '+str(wellsDF.shape[0]))
wellsDF.columns = ['UNIQUE_API']
wellsDF

Number of unique wells in downholeData: 415700


,UNIQUE_API
0,120010000300
1,120010000800
2,120010000900
3,120010001000
4,120010001100
...,...
415695,480590061900
415696,480590064000
415697,480590064200
415698,480590069400


Sort dataset by API Number and Depth of top of record (will be easier to do data analysis with records in the correct order)

In [117]:
downholeData_sorted = downholeData.sort_values(['API_NUMBER','TOP'])
downholeData_sorted.reset_index(inplace=True)
downholeData_sorted
                                

,index,API_NUMBER,TABLE_NAME,FORMATION,THICKNESS,TOP,BOTTOM,INTERPRETATION,CLASS_FLAG,BEDROCK_FLAG
0,2,120010000300,WFORMATIONS,interval drift?,18.0,0.0,18.0,NaN,0.0,False
1,0,120010000300,WFORMATIONS,"Colchester No. 2 coal,Penn. carb",2.0,18.0,20.0,NaN,0.0,False
2,1,120010000300,WFORMATIONS,fire clay at,0.0,20.0,20.0,NaN,0.0,False
3,4,120010000800,WFORMATIONS,interval,175.0,0.0,175.0,NaN,0.0,False
4,3,120010000800,WFORMATIONS,coal No. 2 possibly at,0.0,175.0,175.0,NaN,0.0,False
...,...,...,...,...,...,...,...,...,...,...
2599175,2598491,480590602600,WFORMATIONS,gravel & dirty sand,260.0,19.0,279.0,NaN,0.0,False
2599176,2598493,480590602600,WFORMATIONS,red clay & gravel,240.0,20.0,260.0,NaN,0.0,False
2599177,2598494,480590602600,WFORMATIONS,topsoil & clay,0.0,20.0,20.0,NaN,0.0,False
2599178,2598490,480590602600,WFORMATIONS,fine sand,180.0,60.0,240.0,SAND,1.0,False


Now, for each unique well, find the shallowest record where data goes from non-bedrock to bedrock interval (BEDROCK_FLAG False to BEDROCK_FLAG True)

This will be marked as the top of bedrock, and will be set aside in a new dataframe

Bedrock depth classification type flags are as follows:
- -2: Interval immediately above bedrock interval is unclassified
- -1: At least 1 above-bedrock interval unclassified
- 0: First interval is bedrock
- 1: All intervals classified down to bedrock, some "Start" terms, 
- 2: All intervals classified with certainty

In [118]:
downhole_bedrockDepth = pd.DataFrame(columns=['API_NUMBER','TABLE_NAME','BEDROCK_DEPTH', 'FORMATION', 'BR_CLASS_TYPE'])

#The following flags are used to mark the classification method:
#- 0: Not classified
#- 1: Specific Search Term Match
#- 2: wPermits bedrock top pick
#- 3: Intervals >550' below ground surface
#- 4: Wildcard match (startTerm) - no context
#- 5: Wildcard match (startTerm) - with context
#- Top of well?

for i,w in enumerate(wellsDF['UNIQUE_API']):
    currentWellData = downholeData_sorted[downholeData_sorted['API_NUMBER'] == w].reset_index(drop=True)
    brClassType = 3 #3: iniital state, 0: First interval is bedrock, -1: At least 1 above-bedrock interval unclassified, 1: All intervals classified down to bedrock, some "Start" terms, 2: All intervals classified with certainty
    brClassDetermined = False
    strtTermList = []
    for j, r in enumerate(currentWellData['BEDROCK_FLAG']):
        if not r: #if the current record of the current well has not been classified as bedrock
            if currentWellData.loc[j,'CLASS_FLAG'] == 0: #If current record was not classified and we have not already found a bedrock top
                brClassType = -1 #Note that at least one above-bedrock record is not classified
                brClassDetermined = True #And this will be the final classification type for this well (unless the last record above bedrock is also unclassified)
            elif (currentWellData.loc[j,'CLASS_FLAG']== 4 or currentWellData.loc[j,'CLASS_FLAG']== 5)  and not brClassDetermined:
                brClassType = 1
            elif (brClassType == 3 or brClassType == 2) and currentWellData.loc[j,'CLASS_FLAG']== 1 and not brClassDetermined: #If no other classification types have been made
                brClassType = 2
                strtTermList.append(currentWellData.loc[j,'CLASS_FLAG'])
            #continue looping through the records of current well until we get to the first bedrock interval
        else: #Once we find our first bedrock interval
            if j==0: #Check to see if it is the first interval
                brClassType = 0 #If it is the first interval, note it as such
                brClassDetermined = True #...and make it known we know where bedrock is and how it was determined
            elif currentWellData.loc[j-1,'CLASS_FLAG']==0: #If it is not the first interval, check to see if the record just above bedrock was unclassified
                brClassType = -2 #If it was, note that! This overrides all other bedrock class types
                brClassDetermined = True
            elif brClassType > 0: # If bedrock wasn't the first layer, and no records have been unclassified 
                if 4 in strtTermList or 5 in strtTermList: #If any of the above-bedrock classified records were classified from the startswith search
                    brClassType = 1 #Note that we had at least one such classification
                    brClassDetermined = True
                    
            downhole_bedrockDepth.loc[i, 'API_NUMBER'] = currentWellData.loc[j,'API_NUMBER']
            downhole_bedrockDepth.loc[i, 'TABLE_NAME'] = currentWellData.loc[j,'TABLE_NAME']
            downhole_bedrockDepth.loc[i, 'BEDROCK_DEPTH'] = currentWellData.loc[j,'TOP']
            downhole_bedrockDepth.loc[i, 'FORMATION'] = currentWellData.loc[j,'FORMATION']
            downhole_bedrockDepth.loc[i, 'BR_CLASS_TYPE'] = brClassType
            break #We've found bedrock, we don't need to go on any deeper
downhole_bedrockDepth
        
        

,API_NUMBER,TABLE_NAME,BEDROCK_DEPTH,FORMATION,BR_CLASS_TYPE
2,120010000900,WFORMATIONS,133.0,coal,-2
3,120010001000,WFORMATIONS,137.0,fire clay,-2
4,120010001100,WFORMATIONS,37.0,"limestone, nodular",-2
5,120010001200,WFORMATIONS,35.0,black slate,-2
6,120010001400,DESCRIPTIONS,70.0,"shale, slightly sandy, brownish gray, formatio...",-1
...,...,...,...,...,...
415694,480590061400,WFORMATIONS,13.0,limestone,-1
415695,480590061900,WFORMATIONS,28.0,limestone,0
415697,480590064200,WFORMATIONS,10.0,shale,0
415698,480590069400,WFORMATIONS,36.0,limestone,2


In [119]:
downhole_bedrockDepth['BR_CLASS_TYPE'].value_counts()

 2    65986
 0    60258
-2    60158
-1    25526
 1     8682
Name: BR_CLASS_TYPE, dtype: int64

In [120]:
xyzData = pd.read_csv("\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\headerData__essCols_elev10mDTM.csv",usecols = ['API_NUMBER', 'LONGITUDE','LATITUDE', '10mDTM_ft'])
xyzData

,API_NUMBER,LATITUDE,LONGITUDE,10mDTM_ft
0,120352191101,39.306969,-88.078211,610.581787
1,120352194900,39.188754,-88.186257,577.299255
2,120352204200,39.318753,-88.013434,642.740723
3,120352204300,39.272700,-88.176757,589.783508
4,120352207500,39.178963,-88.077840,580.789856
...,...,...,...,...
634750,120810182400,38.198542,-88.744783,450.715454
634751,120810182500,38.239806,-88.816906,539.966125
634752,120810244500,38.196151,-88.997197,400.415192
634753,120810245100,38.253604,-89.128477,509.683044


In [121]:
downhole_bedrockDepth_XYZ = pd.merge(downhole_bedrockDepth, xyzData.set_index('API_NUMBER'), on='API_NUMBER',how='left')
downhole_bedrockDepth_XYZ['BEDROCK_ELEV_FT'] = downhole_bedrockDepth_XYZ['10mDTM_ft'] - downhole_bedrockDepth_XYZ['BEDROCK_DEPTH']
downhole_bedrockDepth_XYZ['BEDROCK_ELEV_M'] = downhole_bedrockDepth_XYZ['BEDROCK_ELEV_FT'] * 0.3048
downhole_bedrockDepth_XYZ

,API_NUMBER,TABLE_NAME,BEDROCK_DEPTH,FORMATION,BR_CLASS_TYPE,LATITUDE,LONGITUDE,10mDTM_ft,BEDROCK_ELEV_FT,BEDROCK_ELEV_M
0,120010000900,WFORMATIONS,133.0,coal,-2,40.032768,-90.943348,740.961121,607.961121,185.30655
1,120010001000,WFORMATIONS,137.0,fire clay,-2,40.034531,-90.936092,739.937561,602.937561,183.775369
2,120010001100,WFORMATIONS,37.0,"limestone, nodular",-2,40.029328,-90.962715,698.923889,661.923889,201.754401
3,120010001200,WFORMATIONS,35.0,black slate,-2,40.105970,-91.054776,695.992248,660.992248,201.470437
4,120010001400,DESCRIPTIONS,70.0,"shale, slightly sandy, brownish gray, formatio...",-1,40.021306,-91.086866,735.524353,665.524353,202.851823
...,...,...,...,...,...,...,...,...,...,...
220605,480590061400,WFORMATIONS,13.0,limestone,-1,NaN,NaN,NaN,NaN,NaN
220606,480590061900,WFORMATIONS,28.0,limestone,0,NaN,NaN,NaN,NaN,NaN
220607,480590064200,WFORMATIONS,10.0,shale,0,NaN,NaN,NaN,NaN,NaN
220608,480590069400,WFORMATIONS,36.0,limestone,2,NaN,NaN,NaN,NaN,NaN


In [122]:
wPermits_XYZ = pd.merge(wPermits, xyzData.set_index('API_NUMBER'), on='API_NUMBER', how='left')
wPermits_XYZ['BEDROCK_ELEV_FT'] = wPermits_XYZ['10mDTM_ft'] - wPermits_XYZ['BEDROCK_TOP']
wPermits_XYZ['BEDROCK_ELEV_M'] = wPermits_XYZ['BEDROCK_ELEV_FT'] * 0.3048
wPermits_XYZ

,API_NUMBER,WELLTYPE,DEPTH,ISWSPNUM,VERIFIED_BY,VERIFIED_NOTES,VERIFIED_DATE,BEDROCK_TOP,VLQ,VLC,LATITUDE,LONGITUDE,10mDTM_ft,BEDROCK_ELEV_FT,BEDROCK_ELEV_M
0,120012315400,nan,NaN,nan,nan,nan,nan,15.0,nan,nan,40.183949,-90.925546,688.317505,673.317505,205.227175
1,120112265500,,0.0,nan,nan,nan,nan,85.0,nan,nan,41.206931,-89.514952,698.151001,613.151001,186.888425
2,120192630900,MONIT,NaN,nan,JRA,"North of County Road 1800N (Cardinal Rd.), wes...",7/1/2007 0:00:00,314.0,nan,nan,40.142511,-88.344935,706.514526,392.514526,119.638428
3,120192651100,STRAT,NaN,nan,TH,"Update latlongs from A. Stumpf 4/4/16, xy was ...",7/1/2009 0:00:00,424.0,nan,nan,40.100854,-88.229066,755.125366,331.125366,100.927012
4,120192652900,MONIT,NaN,nan,TOH,Iliinois Sustainability Technology Center Illi...,7/1/2009 0:00:00,246.0,nan,nan,40.090426,-88.243053,743.844727,497.844727,151.743073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6056,131290000600,nan,NaN,nan,ACP,Plan and NAIP 2009 imagery,2/11/2013 0:00:00,35.0,5.0,5.0,37.943938,-88.032989,349.701446,314.701446,95.921001
6057,131290000700,nan,NaN,nan,ACP,Plan and NAIP 2009 imagery,2/11/2013 0:00:00,28.0,5.0,5.0,37.943933,-88.035275,329.688324,301.688324,91.954601
6058,131290000800,nan,NaN,nan,ACP,Plan and NAIP 2009 imagery,2/11/2013 0:00:00,23.0,5.0,5.0,37.943806,-88.035180,329.688324,306.688324,93.478601
6059,131290000900,nan,NaN,nan,ACP,Plan and NAIP 2009 imagery,2/11/2013 0:00:00,34.0,5.0,5.0,37.943995,-88.034235,347.566742,313.566742,95.575143


# Export

In [123]:
#Export data 
downhole_bedrockDepth_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\Downhole_BedrockPicks.csv',index_label="ID")
wPermits_XYZ.to_csv('\\\\isgs-sinkhole\\geophysics\\Balikian\\BedrockWellData\\Wells\\ProcessedWellData\\wPermits_BedrockPicks.csv',index_label="ID")